In [92]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import cv2 as cv
import glob
import time
import tensorflow as tf
from tensorflow.python.keras import layers
from keras.layers import Rescaling

%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LayerNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [93]:
dim=100

def Extract_Images_Test(Folder):
    temp=[]
    for filename in glob.glob('C:/Users/antoi/Dropbox/PC/Desktop/Images/' + Folder + '/*.jpg'):
        img=cv.imread(filename)
        temp.append([img,(os.path.basename(Folder)).split('-')[1].replace('_',' ')])
    return(pd.DataFrame(temp,columns=['Image','Race']))


def Extract_Images(Folder):
    temp=[]
    for filename in glob.glob('C:/Users/antoi/Dropbox/PC/Desktop/Images/' + Folder + '/*.jpg'):
        img=cv.imread(filename)
        img=cv.resize(img,dsize=[100, 100])
        temp.append([img,(os.path.basename(Folder)).split('-')[1].replace('_',' ')])
        temp.append([cv.flip(img,0),(os.path.basename(Folder)).split('-')[1].replace('_',' ')])
        temp.append([cv.flip(img,1),(os.path.basename(Folder)).split('-')[1].replace('_',' ')])
        temp.append([cv.flip(img,-1),(os.path.basename(Folder)).split('-')[1].replace('_',' ')])
    return(pd.DataFrame(temp,columns=['Image','Race']))

In [94]:
Folders=os.listdir('C:/Users/antoi/Dropbox/PC/Desktop/Images')
print(len(Folders))

120


# **Extending the models to the whole DataFrame**


In [95]:
num_classes=120

In [96]:
data=pd.DataFrame(columns=['Image','Race','RaceId'],dtype=np.float32)

In [97]:
i=0
for Folder in Folders[:num_classes]:
    tempdata=Extract_Images(Folder)
    data=pd.concat([data,pd.concat([tempdata,pd.DataFrame([i]*(tempdata.shape[0]),columns=['RaceId'])],axis=1)])
    if i%20==0:
        print(i)
    i+=1

0
20
40
60
80
100


In [98]:
data.reset_index(inplace=True)
print(data.shape)

(82320, 4)


In [99]:
class_names=pd.unique(data['Race'])

In [100]:
data['Grayscale']=data['Image'].apply(lambda x : cv.cvtColor(x,cv.COLOR_BGR2GRAY))

In [101]:
data['Grayscale']=data['Grayscale'].apply(lambda x : cv.convertScaleAbs(cv.equalizeHist(x),alpha=1,beta=10))

In [102]:
data_original=data.copy()

In [103]:
for k in range(data['Image'].shape[0]):
    gray=data['Grayscale'][k]
    img=data['Image'][k]
    dst = cv.cornerHarris(gray,2,3,0.04)
    img[dst>0.005*dst.max()]=[255,0,0]
    if k%10000==0:
        print(k)

0
10000
20000
30000
40000
50000
60000
70000
80000


In [104]:
data['Grayscale']=data['Image'].apply(lambda x : cv.cvtColor(x,cv.COLOR_BGR2GRAY))

In [105]:
data['Grayscale']=data['Grayscale'].apply(lambda x : cv.convertScaleAbs(cv.equalizeHist(x),alpha=1,beta=10))

In [106]:
data=pd.concat([data,data_original],axis=0)
data_original=data.copy()

In [107]:
data.reset_index(inplace=True)

print(data.shape)

(164640, 6)


In [108]:
for k in range(data['Image'].shape[0]):
    gray=data['Grayscale'][k]
    img=data['Image'][k]
    sift = cv.xfeatures2d.SIFT_create()
    kp = sift.detect(img, None)
    img=cv.drawKeypoints(gray,kp,img)
    if k%10000==0:
        print(k)

0
10000
20000


KeyboardInterrupt: 

In [ ]:
data['Grayscale']=data['Image'].apply(lambda x : cv.cvtColor(x,cv.COLOR_BGR2GRAY))

In [ ]:
data['Grayscale']=data['Grayscale'].apply(lambda x : cv.convertScaleAbs(cv.equalizeHist(x),alpha=1,beta=10))

In [ ]:
data=pd.concat([data,data_original],axis=0)
data=data.drop(['level_0','index'],axis=1)
data.reset_index(inplace=True)
print(data.shape)

## Pre processing of the data


In [ ]:
train_image,test_image,train_label, test_label = train_test_split(data.loc[:,['Image','Grayscale']],data.loc[:,['Race','RaceId']],stratify=data['RaceId'],test_size=0.1)

In [ ]:
y_counts = pd.DataFrame({
    'data': np.array(['train'] * num_classes + ['test'] * num_classes),
    'class': np.tile(np.arange(num_classes), 2),
    'prop': np.hstack([np.bincount(train_label['RaceId']) / train_label.shape[0],
                         np.bincount(test_label['RaceId']) / test_label.shape[0]])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x='class', y='prop', hue='data', data=y_counts, ax=ax)

In [ ]:
plt.figure(figsize=(10,10))
k=0
for i in train_image.index[:25]:
    plt.subplot(5,5,k+1)
    k+=1
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_image['Image'][i])
    plt.xlabel(train_label['Race'][i])
plt.show()

We normalize the data:

In [ ]:
train_images = train_image['Image']
test_images = test_image['Image']

train_labels=train_label['RaceId']
test_labels=test_label['RaceId']

The scale of grey is now from 0 to 1.

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

###**Creation of a validation set**

We further split the train set into a train and validation set. The validation set will be 20% from the original train set, therefore the split will be train/validation of 0.8/0.2.

The actual training set will be divided into two groups:
* the training set
* the validation set

In [ ]:

train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=2020)

In [ ]:
train_id=train_images.index
valid_id=valid_images.index
test_id=test_images.index

In [ ]:
partition = {'train' : [str(i) for i in train_id], 'validation' : [str(i) for i in valid_id]}

labels={}
for i in train_id:
    labels[str(i)]=train_labels[i]
for i in valid_id:
    labels[str(i)]=valid_labels[i]


### We define a class used not to overflow our GPU.

In [ ]:
batch_size=256
epochs=200

In [ ]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=batch_size, dim=input_shape, n_classes=num_classes, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = data['Image'][int(ID)]

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
train_images=np.asarray([i.astype(np.float32) for i in train_images])

valid_images=np.asarray([i.astype(np.float32) for i in valid_images])

test_images=np.asarray([i.astype(np.float32) for i in test_images])

input_shape = train_images.shape[1:]

In [ ]:
test_labels = to_categorical(test_labels, num_classes)
test_labels=np.asarray([i.astype(np.float32) for i in test_labels])


In [ ]:
def create_neural_network(MaxPoolNum=3,PoolSizeList=[(3,3),(3,3),(2,2)],ConvList=[2,2,3],KerSizes=[[(3,3),(3,3)],[(3,3),(3,3)],[(3,3),(3,3),(3,3)]],ConvArgList=[[32,32],[64,64],[128,128,256]],optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],rescaling=True,input_shape=input_shape,dropout_rate=0.75):
    model_temp = Sequential()
    if rescaling:
        model_temp.add(Rescaling(1./255, input_shape=input_shape))
    for k in range(MaxPoolNum):
        for i in range(ConvList[k]):
            if k==0 and i==0:
                model_temp.add(Conv2D(ConvArgList[k][i],kernel_size=KerSizes[k][i],activation='relu', input_shape=input_shape))
            else:
                model_temp.add(Conv2D(ConvArgList[k][i],kernel_size=KerSizes[k][i],activation='relu'))
        if k==0:
            model_temp.add(LayerNormalization())
        model_temp.add(MaxPooling2D(pool_size=PoolSizeList[k]))


    model_temp.add(Flatten())
    model_temp.add(Dense(num_classes*5, activation='relu'))
    model_temp.add(Dropout(dropout_rate))
    model_temp.add(Dense(num_classes, activation='softmax'))

    model_temp.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model_temp


In [ ]:
#Generators

training_generator = DataGenerator(partition['train'], labels)
validation_generator = DataGenerator(partition['validation'], labels)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)

### Run the models

First model

In [ ]:
time0=time.time()
wholemodel_cnn = create_neural_network(MaxPoolNum=3,PoolSizeList=[3,2,2],ConvList=[1,1,1],
                                        KerSizes=[[10],[8],[6]],ConvArgList=[[16],[32],[32]],
                                        optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],
                                        rescaling=True,input_shape=input_shape,dropout_rate=0.4)

wholemodel_cnn.summary()

In [ ]:
# Run the train
wholehistory_cnn = wholemodel_cnn.fit(training_generator,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])

In [ ]:
wholescore_cnn = wholemodel_cnn.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_cnn[0])
print('Test accuracy:', wholescore_cnn[1])
time1=time.time()-time0

Second model

In [ ]:
time0=time.time()
wholemodel_cnn2 = create_neural_network(MaxPoolNum=3,PoolSizeList=[(3,3),(3,3),(3,3)],ConvList=[2,2,1],
                                        KerSizes=[[(3,3),(3,3)],[(4,4),(3,3)],[(3,3)]],ConvArgList=[[32,32],[64,64],[256]],
                                        optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],
                                        rescaling=True,input_shape=input_shape,dropout_rate=0.7)


wholemodel_cnn2.summary()

In [ ]:
# Run the train
wholehistory_cnn2 = wholemodel_cnn2.fit(training_generator,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])

In [ ]:
wholescore_cnn2 = wholemodel_cnn2.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_cnn2[0])
print('Test accuracy:', wholescore_cnn2[1])
time1=time.time()-time0

# Re-training du VGG16

In [ ]:
from keras.applications import VGG16
from keras import models

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False ## Not trainable weights


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(num_classes*10, activation='relu')
dense_layer_2 = layers.Dense(num_classes*5, activation='relu')
dropout_layer = layers.Dropout(0.25)
prediction_layer = layers.Dense(num_classes, activation='softmax')


wholemodel = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dropout_layer,
    dense_layer_2,
    prediction_layer
])

wholemodel.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

wholehistory_model=wholemodel.fit(training_generator,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])

In [ ]:
wholescore_model = wholemodel.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_model[0])
print('Test accuracy:', wholescore_model[1])

In [ ]:
valid_set= (valid_images,valid_labels)



In [ ]:
#Loss
print("1st CNN MODEL: ")
history_loss(wholehistory_cnn)
print("2nd CNN MODEL:")
history_loss(wholehistory_cnn2)
print("VGG16 MODEL: ")
history_loss(wholehistory_model)


In [ ]:
# Accuracy
print("1st CNN MODEL: ")
history_accuracy(wholehistory_cnn)
print("2nd CNN MODEL:")
history_accuracy(wholehistory_cnn2)
print("VGG16 MODEL: ")
history_accuracy(wholehistory_model)



In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
CNN = ['CNN 2', 'VGG16']
Test_Accuracy = [wholescore_cnn2[1],wholescore_model[1]]
ax.bar(CNN,Test_Accuracy)
plt.show()

In [ ]:
def plot_prediction_images(prediction):
    num_rows = 5
    num_cols = 3
    num_images = num_rows*num_cols
    plt.figure(figsize=(2*2*num_cols, 2*num_rows))
    for i in range (num_images):
        plt.subplot(num_rows, 2*num_cols, 2*i+1)
        plot_image(i, prediction[i], test_labels, test_images)
        plt.subplot(num_rows, 2*num_cols, 2*i+2)
        plot_value_array(i, prediction[i], test_labels)
    plt.tight_layout()
    plt.show()

In [ ]:
def create_neural_network(MaxPoolNum=3,PoolSizeList=[(3,3),(3,3),(2,2)],ConvList=[2,2,3],KerSizes=[[(3,3),(3,3)],[(3,3),(3,3)],[(3,3),(3,3),(3,3)]],ConvArgList=[[32,32],[64,64],[128,128,256]],optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],rescaling=True,input_shape=input_shape,dropout_rate=0.75):
    model_temp = Sequential()
    if rescaling:
        model_temp.add(Rescaling(1./255, input_shape=input_shape))
    for k in range(MaxPoolNum):
        for i in range(ConvList[k]):
            if k==0 and i==0:
                model_temp.add(Conv2D(ConvArgList[k][i],kernel_size=KerSizes[k][i],activation='relu', input_shape=input_shape))
            else:
                model_temp.add(Conv2D(ConvArgList[k][i],kernel_size=KerSizes[k][i],activation='relu'))
        if k==0:
            model_temp.add(LayerNormalization())
        model_temp.add(MaxPooling2D(pool_size=PoolSizeList[k]))


    model_temp.add(Flatten())
    model_temp.add(Dense(num_classes*5, activation='relu'))
    model_temp.add(Dropout(dropout_rate))
    model_temp.add(Dense(num_classes, activation='softmax'))

    model_temp.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model_temp